In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns  
import tensorflow.keras as keras

from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.math import confusion_matrix
from sklearn.metrics import balanced_accuracy_score
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder()
from scipy.stats import bernoulli
from tqdm import tqdm

from google.colab import drive
drive.mount('/content/drive')

gpu_info = !nvidia-smi -L
gpu_info = '\n'.join(gpu_info)
print(gpu_info)

Mounted at /content/drive
GPU 0: Tesla P100-PCIE-16GB (UUID: GPU-1daaadef-292f-1eec-2d7e-ff0f195392b7)


In [3]:
#Function to create a model

def create_model(input_shape,
                 n_classes,
                 loss,
                 units_LSTM=[256,128,64],
                 units_Dense=[64,32],
                 dropout_LSTM=0.2,
                 dropout_Dense=0.15,
                 learning_rate=0.001
                 ):
    """
    Creates a LSTM model with 3 LSTM layers and 3 dense layers (the last one is output layer)
    """
    assert units_LSTM[2] == units_Dense[0], "Dimensions of 3rd LSTM Layer and 1st Dense Layer should fit"

    model = keras.Sequential()
    # 2 LSTM layers
    model.add(keras.layers.LSTM(units=units_LSTM[0],
                                input_shape=input_shape,
                                return_sequences=True,
                                dropout=dropout_LSTM
                                ))

    model.add(keras.layers.LSTM(units=units_LSTM[1],
                                input_shape=input_shape,
                                return_sequences=True,
                                dropout=dropout_LSTM
                                ))

    model.add(keras.layers.LSTM(units_LSTM[2]))

    # dense layers
    model.add(keras.layers.Dense(units=units_Dense[0], activation='relu'))
    model.add(keras.layers.Dropout(dropout_Dense))

    model.add(keras.layers.Dense(units_Dense[1], activation='relu'))
    model.add(keras.layers.Dropout(dropout_Dense))

    # output layer
    model.add(keras.layers.Dense(n_classes, activation='softmax'))

    #compile the model
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
                  loss=loss,
                  metrics=['accuracy'])
    
    return model

def get_2d_conv_model(input_shape,
                 n_classes,
                 loss,
                 dropout_Dense=0.15,
                 learning_rate=0.001,
                 conv_lst=[512, 256, 128],
                 dens_lst=[512, 256],
                 kernels = 4):
    
    model = keras.Sequential()
    inp = keras.layers.Input(shape=(input_shape[0], input_shape[1], 1))
    model.add(keras.layers.Convolution2D(conv_lst[0], (kernels, kernels), padding="same"))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Activation("relu"))
    model.add(keras.layers.MaxPool2D())
    model.add(keras.layers.Dropout(dropout_Dense))

    
    model.add(keras.layers.Convolution2D(conv_lst[1], (kernels, kernels*2), padding="same"))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Activation("relu"))
    model.add(keras.layers.MaxPool2D())
    model.add(keras.layers.Dropout(dropout_Dense))

    
    model.add(keras.layers.Convolution2D(conv_lst[1], (kernels,kernels*2), padding="same"))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Activation("relu"))
    model.add(keras.layers.MaxPool2D())
    model.add(keras.layers.Dropout(dropout_Dense))

    
    model.add(keras.layers.Convolution2D(conv_lst[2], (kernels,kernels*2), padding="same"))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Activation("relu"))
    model.add(keras.layers.MaxPool2D())
    model.add(keras.layers.Dropout(dropout_Dense))

    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(512))
    model.add(keras.layers.Dropout(dropout_Dense))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Activation("relu"))

    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(256))
    model.add(keras.layers.Dropout(dropout_Dense))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Activation("relu"))

    model.add(keras.layers.Dense(n_classes, activation='softmax'))

    model.compile(optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
                  loss=loss,
                  metrics=['accuracy'])
    
    return model
    

In [4]:
X_raw=np.load('/content/drive/MyDrive/Pro2/fet.npy')
y_raw=pd.read_csv('/content/drive/MyDrive/Pro2/y_raw_clean.csv')
noice_egz = ["zero", "one","two", "three",
             "four", "five", "six", "seven",
             "eight", "nine","bed", "bird",
             "cat", "dog", "happy", "house",
             "marvin", "sheila", "tree", "wow"]

y_raw['label'] = np.where(y_raw['label'].isin(noice_egz), "unknown", y_raw['label'])
encoded = pd.read_csv('/content/drive/MyDrive/Pro2/models/label_encodings.csv')

y_all = y_raw.merge(encoded, left_on='label', right_on='name',how="left")['index']
X_raw = X_raw.reshape((65120, 20, 98, 1))
X_train, X_test, y_train, y_test=train_test_split(X_raw,y_all,stratify=y_all,test_size=0.25)
input_shape = (X_train.shape[1], X_train.shape[2]) # 20,98
n_classes=len(np.unique(y_train)) #12

In [31]:
dropout_=np.arange(start=0.2,stop=0.5,step=0.1)
learning_rate=[10.0**(-i) for i in np.arange(start=2,stop=5,step=1)]
units_=[[128,128,128],[512,256,128]]
kernels = [5, 7]
n_epochs=7
callback=EarlyStopping(
    monitor="val_loss",
    min_delta=1e-4,
    patience=3,
    verbose=0,
    restore_best_weights=True,
)

In [ ]:
ans_list = []

In [33]:
#from pandas.core.dtypes.base import E

for kernel in tqdm(kernels):
    for dropout_id,dropout in enumerate(dropout_):
        for lr_id,lr in enumerate(learning_rate):
            for unit_id,unit in enumerate(units_):
                if bernoulli.rvs(0.15):
                  model=get_2d_conv_model(input_shape=input_shape,
                  n_classes=n_classes,
                  conv_lst=unit,
                  dropout_Dense=dropout,
                  learning_rate=lr,
                  kernels=kernel,
                  loss="sparse_categorical_crossentropy")

                  print("learning: drop: {} lr: {}, kernel:{}, unit:{}".format(dropout, lr, kernel, unit))
                  history=model.fit(X_train, y_train, validation_data=(X_test, y_test), batch_size=128,callbacks=callback,epochs=n_epochs,verbose=0)
                  ans_list.append((dropout,lr,unit,history.history['accuracy'][-1], kernel))
                  print(str(dropout_id)+str(lr_id)+str(unit_id)+" done\n")
                  keras.backend.clear_session()
                else:
                  pass

  0%|          | 0/2 [00:00<?, ?it/s]

learning: drop: 0.2 lr: 0.01, kernel:5, unit:[512, 256, 128]


 50%|█████     | 1/2 [13:24<13:24, 804.79s/it]

001 done

learning: drop: 0.2 lr: 0.01, kernel:7, unit:[512, 256, 128]
001 done

learning: drop: 0.30000000000000004 lr: 0.01, kernel:7, unit:[512, 256, 128]
101 done

learning: drop: 0.30000000000000004 lr: 0.0001, kernel:7, unit:[512, 256, 128]


100%|██████████| 2/2 [1:14:43<00:00, 2241.89s/it]

121 done

